# Planning: planning.py; chapters 10-11

This notebook describes the [planning.py](https://github.com/aimacode/aima-python/blob/master/planning.py) module, which covers Chapters 10 (Classical Planning) and  11 (Planning and Acting in the Real World) of *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*. See the [intro notebook](https://github.com/aimacode/aima-python/blob/master/intro.ipynb) for instructions.

We'll start by looking at `PDDL` and `Action` data types for defining problems and actions. Then, we will see how to use them by trying to plan a trip from *Sibiu* to *Bucharest* across the familiar map of Romania, from [search.ipynb](https://github.com/aimacode/aima-python/blob/master/search.ipynb). Finally, we will look at the implementation of the GraphPlan algorithm.

The first step is to load the code:

In [1]:
from planning import *

To be able to model a planning problem properly, it is essential to be able to represent an Action. Each action we model requires at least three things:
* preconditions that the action must meet
* the effects of executing the action
* some expression that represents the action

Planning actions have been modelled using the `Action` class. Let's look at the source to see how the internal details of an action are implemented in Python.

In [2]:
%psource Action

It is interesting to see the way preconditions and effects are represented here. Instead of just being a list of expressions each, they consist of two lists - `precond_pos` and `precond_neg`. This is to work around the fact that PDDL doesn't allow for negations. Thus, for each precondition, we maintain a seperate list of those preconditions that must hold true, and those whose negations must hold true. Similarly, instead of having a single list of expressions that are the result of executing an action, we have two. The first (`effect_add`) contains all the expressions that will evaluate to true if the action is executed, and the the second (`effect_neg`) contains all those expressions that would be false if the action is executed (ie. their negations would be true).

The constructor parameters, however combine the two precondition lists into a single `precond` parameter, and the effect lists into a single `effect` parameter.

The `PDDL` class is used to represent planning problems in this module. The following attributes are essential to be able to define a problem:
* a goal test
* an initial state
* a set of viable actions that can be executed in the search space of the problem

View the source to see how the Python code tries to realise these.

In [3]:
%psource PDDL

The `initial_state` attribute is a list of `Expr` expressions that forms the initial knowledge base for the problem. Next, `actions` contains a list of `Action` objects that may be executed in the search space of the problem. Lastly, we pass a `goal_test` function as a parameter - this typically takes a knowledge base as a parameter, and returns whether or not the goal has been reached.

Now lets try to define a planning problem using these tools. Since we already know about the map of Romania, lets see if we can plan a trip across a simplified map of Romania.

Here is our simplified map definition:

In [4]:
from utils import *
# this imports the required expr so we can create our knowledge base

knowledge_base = [
    expr("Connected(Bucharest,Pitesti)"),
    expr("Connected(Pitesti,Rimnicu)"),
    expr("Connected(Rimnicu,Sibiu)"),
    expr("Connected(Sibiu,Fagaras)"),
    expr("Connected(Fagaras,Bucharest)"),
    expr("Connected(Pitesti,Craiova)"),
    expr("Connected(Craiova,Rimnicu)")
    ]

Let us add some logic propositions to complete our knowledge about travelling around the map. These are the typical symmetry and transitivity properties of connections on a map. We can now be sure that our `knowledge_base` understands what it truly means for two locations to be connected in the sense usually meant by humans when we use the term.

Let's also add our starting location - *Sibiu* to the map.

In [5]:
knowledge_base.extend([
     expr("Connected(x,y) ==> Connected(y,x)"),
     expr("Connected(x,y) & Connected(y,z) ==> Connected(x,z)"),
     expr("At(Sibiu)")
    ])

We now have a complete knowledge base, which can be seen like this:

In [6]:
knowledge_base

[Connected(Bucharest, Pitesti),
 Connected(Pitesti, Rimnicu),
 Connected(Rimnicu, Sibiu),
 Connected(Sibiu, Fagaras),
 Connected(Fagaras, Bucharest),
 Connected(Pitesti, Craiova),
 Connected(Craiova, Rimnicu),
 (Connected(x, y) ==> Connected(y, x)),
 ((Connected(x, y) & Connected(y, z)) ==> Connected(x, z)),
 At(Sibiu)]

We now define possible actions to our problem. We know that we can drive between any connected places. But, as is evident from [this](https://en.wikipedia.org/wiki/List_of_airports_in_Romania) list of Romanian airports, we can also fly directly between Sibiu, Bucharest, and Craiova.

We can define these flight actions like this:

In [7]:
#Sibiu to Bucharest
precond_pos = [expr('At(Sibiu)')]
precond_neg = []
effect_add = [expr('At(Bucharest)')]
effect_rem = [expr('At(Sibiu)')]
fly_s_b = Action(expr('Fly(Sibiu, Bucharest)'), [precond_pos, precond_neg], [effect_add, effect_rem])

#Bucharest to Sibiu
precond_pos = [expr('At(Bucharest)')]
precond_neg = []
effect_add = [expr('At(Sibiu)')]
effect_rem = [expr('At(Bucharest)')]
fly_b_s = Action(expr('Fly(Bucharest, Sibiu)'), [precond_pos, precond_neg], [effect_add, effect_rem])

#Sibiu to Craiova
precond_pos = [expr('At(Sibiu)')]
precond_neg = []
effect_add = [expr('At(Craiova)')]
effect_rem = [expr('At(Sibiu)')]
fly_s_c = Action(expr('Fly(Sibiu, Craiova)'), [precond_pos, precond_neg], [effect_add, effect_rem])

#Craiova to Sibiu
precond_pos = [expr('At(Craiova)')]
precond_neg = []
effect_add = [expr('At(Sibiu)')]
effect_rem = [expr('At(Craiova)')]
fly_c_s = Action(expr('Fly(Craiova, Sibiu)'), [precond_pos, precond_neg], [effect_add, effect_rem])

#Bucharest to Craiova
precond_pos = [expr('At(Bucharest)')]
precond_neg = []
effect_add = [expr('At(Craiova)')]
effect_rem = [expr('At(Bucharest)')]
fly_b_c = Action(expr('Fly(Bucharest, Craiova)'), [precond_pos, precond_neg], [effect_add, effect_rem])

#Craiova to Bucharest
precond_pos = [expr('At(Craiova)')]
precond_neg = []
effect_add = [expr('At(Bucharest)')]
effect_rem = [expr('At(Craiova)')]
fly_c_b = Action(expr('Fly(Craiova, Bucharest)'), [precond_pos, precond_neg], [effect_add, effect_rem])

And the drive actions like this.

In [8]:
#Drive
precond_pos = [expr('At(x)')]
precond_neg = []
effect_add = [expr('At(y)')]
effect_rem = [expr('At(x)')]
drive = Action(expr('Drive(x, y)'), [precond_pos, precond_neg], [effect_add, effect_rem])

Finally, we can define a a function that will tell us when we have reached our destination, Bucharest.

In [9]:
def goal_test(kb):
    return kb.ask(expr("At(Bucharest)"))

Thus, with all the components in place, we can define the planning problem.

In [10]:
prob = PDDL(knowledge_base, [fly_s_b, fly_b_s, fly_s_c, fly_c_s, fly_b_c, fly_c_b, drive], goal_test)